In [32]:
!python3 -m pip install --upgrade pip
!pip3 install -qU langgraph
!pip3 install langgraph-checkpoint-sqlite
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List, Optional
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
import os
import requests
import json
from openai import OpenAI
import httpx
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain.agents import initialize_agent, AgentType, load_tools
from langchain_core.tools import Tool
from pydantic import BaseModel, ValidationError
#from langchain_core.pydantic_v1 import BaseModel
from langchain.tools import tool
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate
from langgraph.checkpoint.sqlite import SqliteSaver
from tavily import TavilyClient
from langchain_community.tools.tavily_search import TavilySearchResults
from IPython.display import Image
import re

# connect to tavily search tool
os.environ['TAVILY_API_KEY']="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"
tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])
#tool = TavilySearchResults(max_results=2)

#define agent state
class AgentState(TypedDict):
    task: str
    lnode: str
    plan: str
    draft: str
    critique: str
    content: List[str]
    queries: List[str]
    revision_number: int
    max_revisions: int
    count: Annotated[int, operator.add]


#define and configure the model
# configure model
httpx_client = httpx.Client(http2=True, verify=False, timeout=10.0)

vcapservices = os.getenv('VCAP_SERVICES')
services = json.loads(vcapservices)

def is_chatservice(service):
    return service["name"] == "gen-ai-qwen3-ultra"

chat_services = filter(is_chatservice, services["genai"])
chat_credentials = list(chat_services)[0]["credentials"]

model = ChatOpenAI(temperature=0, model=chat_credentials["model_name"], base_url=chat_credentials["api_base"], api_key=chat_credentials["api_key"], http_client=httpx_client)


#define prompts
PLAN_PROMPT = """
You are an expert writer tasked with writing a high level outline of an eassy. \
Write such an outline for the user provided topic. Give an outline of eassy along \
with any relevant notes or instructions for the sections.
"""

WRITER_PROMPT = """
You are an eassy assistant tasked with writing excellent 5-paragraph eassys. \
Generate the best eassy possible for the user's request and the initial outline. \
If the user provides critique, respond with a revised version of ypur previous attempts. \

--------

{content}"""

REFLECTION_PROMPT = """
You are a teacher grading an eassy submission. \
Generate critique and recommendations for the user's submission. \
Provide detailed recommendations, including requests for length, depth, style, etc.
"""

RESEARCH_PLAN_PROMPT = """
You are a researcher charged with providing information that can \
be used when writing the following eassy. Generate a list of search queries that will gather \
any relevant information. Only generate 3 queries max
"""

RESEARCH_CRITIQUE_PROMPT = """
You are a researcher charged with providing information that can \
be used when making any requested revisions (as oulined below). \
Generate a list of search queries that will gather any relevant information. \
Only generate 3 queries max.
"""
    
def extract_json(text):
    # Remove unwanted tags like <think> and <speak>
    cleaned_text = re.sub(r'<\/?[\w\d]+>', '', text).strip()

    # Now try to extract the JSON part using regex
    match = re.search(r'\{.*\}', cleaned_text, re.DOTALL)
    if not match:
        raise ValueError("No JSON object found in response")
    return json.loads(match.group(0))
    
class Queries(BaseModel):
    queries: List[str]

#implement nodes
def plan_node(state: AgentState):
    messages = [
        SystemMessage(content=PLAN_PROMPT),
        HumanMessage(content=state["task"])
    ]
    response = model.invoke(messages)
    return {"plan": response.content}
    
def research_plan_node(state: AgentState):
    raw_response = model.invoke([
        SystemMessage(content=RESEARCH_PLAN_PROMPT),
        HumanMessage(content=state['task'])
    ])

    try:
        json_data = extract_json(raw_response.content if hasattr(raw_response, 'content') else str(raw_response))
        queries = Queries.model_validate(json_data)
    except Exception as e:
        print("Error parsing JSON from model output:", e)
        return {"content": state.get('content', [])}

    content = state.get('content', [])
    for q in queries.queries:
        response = tavily.search(query=q, max_results=2)
        for r in response['results']:
            content.append(r['content'])

    return {"content": content}
def generation_node(state: AgentState):
    content = "\n\n".join(["content"] or [])
    user_message = HumanMessage(content=f"{state['task']}\n\nHere is my plan:\n\n{state['plan']}")
    messages = [
        SystemMessage(content=WRITER_PROMPT.format(content=content)),
        user_message,
    ]
    response = model.invoke(messages)
    return {
        "draft": response.content,
        "revision_number": state.get("revision_number", 1) + 1,
    }
def reflection_node(state: AgentState):
    messages = [
        SystemMessage(content=REFLECTION_PROMPT),
        HumanMessage(content=state['draft']),
    ]
    response = model.invoke(messages)
    return {"critique": response.content}

def research_critique_node(state: AgentState):
    try:
        # Run the model
        raw_response = model.invoke([
            SystemMessage(content=RESEARCH_CRITIQUE_PROMPT),
            HumanMessage(content=state['critique'])
        ])
        
        # Extract JSON from model output (if Qwen adds extra tokens)
        text_output = raw_response.content if isinstance(raw_response, AIMessage) else str(raw_response)
        parsed = extract_json(text_output)
        queries = Queries.model_validate(parsed)

    except Exception as e:
        print("Error during model invocation or JSON parsing:", e)
        return {"content": state.get("content", [])}

    content = state.get("content", [])
    for q in queries.queries:
        try:
            response = tavily.search(query=q, max_results=2)
            for r in response.get("results", []):
                content.append(r.get("content", ""))
        except Exception as e:
            print(f"Search failed for query '{q}': {e}")
    
    return {"content": content}

def should_continue(state):
    if state["revision_number"] > state["max_revisions"]:
        return END
    return "reflect"


#build graph
builder = StateGraph(AgentState)
builder.add_node("planner", plan_node)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_node("research_plan", research_plan_node)
builder.add_node("research_critique", research_critique_node)

#set entry point
builder.set_entry_point("planner")

#define conditional edges
builder.add_conditional_edges(
    "generate", 
    should_continue, 
    {END: END, "reflect": "reflect"}
)

#define edges
builder.add_edge("planner", "research_plan")
builder.add_edge("research_plan", "generate")

builder.add_edge("reflect", "research_critique")
builder.add_edge("research_critique", "generate")



#print graph
#Image(graph.get_graph().draw_png())

thread = {"configurable": {"thread_id": "1"}}
with SqliteSaver.from_conn_string(":memory:") as checkpointer:
    graph = builder.compile(checkpointer=checkpointer)
    for s in graph.stream({
        'task': "what is the difference between langchain and langsmith",
        "max_revisions": 2,
        "revision_number": 1,
    }, thread):
        print(s)



{'planner': {'plan': '<think>\nOkay, the user is asking about the difference between LangChain and LangSmith. Let me start by recalling what each of these tools does. \n\nFirst, LangChain. I remember that LangChain is a framework for building applications using language models. It provides tools to connect different components, like data sources, models, and outputs. It\'s used for things like creating chains of operations, integrating with various models, and handling data. So, maybe I should explain that LangChain is more about the framework and tools for building applications.\n\nThen there\'s LangSmith. I think LangSmith is a platform or tool that\'s part of the LangChain ecosystem. Wait, but I need to confirm. Maybe LangSmith is a specific tool within LangChain\'s ecosystem. Or perhaps they are separate but related. I should check if there\'s any official information. \n\nWait, I might be mixing up some details. Let me think. LangChain is a framework, and LangSmith could be a tool